In [1]:
import pandas as pd
import numpy as np 
import re
import nltk
#from transformers import BertTokenizer

### Create a setence level data set using the filename, speaker, and timestamp as labels 

In [2]:
file_path = '/Users/steffenerickson/Box Sync/ECR Observation Data/2023-2024 Final Data/Quant Team - Results'
df = pd.read_csv(f'{file_path}/transcripts.csv')

In [3]:
df_lines_split = df['text'].str.split(r'\\n').apply(pd.Series).stack().to_frame('line_str')
df_lines_split.reset_index(drop=True, inplace=True)
df_lines_split['filename'] = df['filename'].repeat(df_lines_split.shape[0] // len(df)).reset_index(drop=True)
df_cleaned = df_lines_split[df_lines_split['line_str'].str.strip() != ""]

In [4]:
def is_teacher_timestamp(text):
    return bool(re.search(r'teacher \d+.*\d+:\d+', text, re.IGNORECASE))
def is_transcribed_by_otter(text):
    return bool(re.search(r'transcribed by https://otter\.ai', text, re.IGNORECASE))

In [5]:
df_cleaned = df_cleaned.copy()
df_cleaned.loc[:, 'is_teacher_timestamp'] = df_cleaned['line_str'].apply(is_teacher_timestamp)
df_cleaned.loc[:, 'firstrow'] = df_cleaned.groupby('filename')['is_teacher_timestamp'].transform(
    lambda x: x.idxmax() == x.index).fillna(False).astype(int)
df_cleaned.loc[:, 'is_transcribed_by_otter'] = df_cleaned['line_str'].apply(is_transcribed_by_otter)
df_cleaned.loc[:, 'lastrow'] = df_cleaned['is_transcribed_by_otter'].apply(lambda x: 1 if x else 0)

In [6]:
def keep_between_ones(group):
    start_idx = group['firstrow'].idxmax()  # First occurrence of 1 in the second column
    end_idx = group['lastrow'].idxmax()    # First occurrence of 1 in the third column
    return group.loc[start_idx:end_idx-1]
df_chopped = df_cleaned.groupby('filename', group_keys=False).apply(keep_between_ones)
df_chopped = df_chopped.drop(['firstrow', 'lastrow','is_teacher_timestamp','is_transcribed_by_otter'], axis=1)

In [7]:
# Define a function to classify the rows based on the 'line_str'
def classify_line_str(line):
    if re.match(r'Teacher.*\d{2}:\d{2}', line):
        return 1
    elif re.match(r'.*\d{2}:\d{2}', line):
        return 2
    else:
        return np.nan
df_chopped['classification'] = df_chopped['line_str'].apply(classify_line_str)

In [8]:
# Define a function to extract the person (text before the timestamp) and timestamp
def extract_person_and_timestamp(line):
    # Use regex to match the format "[text] [timestamp]"
    match = re.search(r'(.+?)\s+(\d{2}:\d{2})', line)  # Adjusted to find any text before the timestamp
    if match:
        return match.group(1).strip(), match.group(2)
    else:
        return np.nan, np.nan
df_chopped['person'], df_chopped['timestamp'] = zip(*df_chopped.apply(
    lambda row: extract_person_and_timestamp(row['line_str']) if pd.notna(row['classification']) else (np.nan, np.nan), axis=1))

In [9]:
df_chopped['keep'] = df_chopped['classification'].apply(lambda x: 1 if pd.isna(x) else 0)
df_chopped['classification'] = df_chopped['classification'].ffill()
df_chopped['person'] = df_chopped['person'].ffill()
df_chopped['timestamp'] = df_chopped['timestamp'].ffill()
df_filtered = df_chopped[df_chopped['keep'] == 1]
df_filtered = df_filtered.drop(['keep'], axis=1)
df_filtered.reset_index(drop=True, inplace=True)

In [10]:
df_sentences = df_filtered.line_str.apply(lambda x: pd.Series(nltk.sent_tokenize(x))).stack().to_frame('sent_str')
df_sentences = df_sentences.reset_index(level=1, drop=True)
variables = ['filename', 'classification', 'person', 'timestamp','line_str']
for var in variables:
    df_sentences[var] = df_filtered[var].repeat(df_sentences.groupby(level=0).size()).values

In [11]:
df_speaker = df_sentences[df_sentences['person'].str.contains('speaker', case=False, na=False)]

In [12]:
# Apply the conditions to locate the rows to be updated
condition = (
    (df_sentences['filename'] == "01_0101_G4_L03_transcript.txt") & 
    (df_sentences['person'].str.contains('speaker', case=False, na=False))
)
# Update the 'person' column
df_sentences.loc[condition, 'person'] = "Teacher 01_0101"
# Update the 'classification' column to 1 for the same rows
df_sentences.loc[condition, 'classification'] = 1

### Tokenize Sentences 

In [13]:
keep_whitespace = True
if keep_whitespace:
    # Return a tokenized copy of text
    # using NLTK's recommended word tokenizer.
    df_tokens = df_sentences.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.word_tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
else:
    # Tokenize a string on whitespace (space, tab, newline).
    # In general, users should use the string ``split()`` method instead.
    # Returns fewer tokens.
    df_tokens = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
df_tokens['pos'] = df_tokens.pos_tuple.apply(lambda x: x[1])
df_tokens['token_str'] = df_tokens.pos_tuple.apply(lambda x: x[0])
df_tokens['term_str'] = df_tokens.token_str.str.lower().str.replace(r"\W+", "", regex=True)
df_tokens['pos_group'] = df_tokens.pos.str[:2]

In [14]:
variables = ['filename', 'classification', 'person', 'timestamp','sent_str','line_str']
repeat_counts = df_sentences['sent_str'].apply(lambda x: len(nltk.word_tokenize(x)))
for var in variables:
    df_tokens[var] = np.repeat(df_sentences[var].values, repeat_counts)

In [15]:
df_tokens = df_tokens[df_tokens.term_str != '']

In [16]:
df_tokens.head(50)

pos_tuple   pos token_str  term_str pos_group  \
0 0       (Nope, NN)    NN      Nope      nope        NN   
  0   (Alright, NNP)   NNP   Alright   alright        NN   
  2       (what, WP)    WP      what      what        WP   
  3       (are, VBP)   VBP       are       are        VB   
  4      (they, PRP)   PRP      they      they        PR   
  0       (What, WP)    WP      What      what        WP   
  1   (planets, NNS)   NNS   planets   planets        NN   
  2      (have, VBP)   VBP      have      have        VB   
  3     (rings, NNS)   NNS     rings     rings        NN   
  0        (Yep, NN)    NN       Yep       yep        NN   
  0        (One, CD)    CD       One       one        CD   
  1      (more, JJR)   JJR      more      more        JJ   
1 0       (This, DT)    DT      This      this        DT   
  1        (is, VBZ)   VBZ        is        is        VB   
  2       (easy, JJ)    JJ      easy      easy        JJ   
2 0        (All, DT)    DT       All       all        DT   
  1      (right, NN)    NN     right     right        NN   
  3        (it, PRP)   PRP        it        it        PR   
  4        ('s, VBZ)   VBZ        's         s        VB   
  5        (not, RB)    RB       not       not        RB   
  6        (gon, VB)    VB       gon       gon        VB   
  7         (na, RB)    RB        na        na        RB   
  8         (be, VB)    VB        be        be        VB   
  9        (one, CD)    CD       one       one        CD   
  10        (of, IN)    IN        of        of        IN   
  11     (our, PRP$)  PRP$       our       our        PR   
  12     (inner, JJ)    JJ     inner     inner        JJ   
  13  (planets, NNS)   NNS   planets   planets        NN   
  0       (Say, NNP)   NNP       Say       say        NN   
  1       (that, IN)    IN      that      that        IN   
  3        (it, PRP)   PRP        it        it        PR   
  4        ('s, VBZ)   VBZ        's         s        VB   
  5        (not, RB)    RB       not       not        RB   
  6        (gon, VB)    VB       gon       gon        VB   
  7         (na, RB)    RB        na        na        RB   
  8         (be, VB)    VB        be        be        VB   
  9        (one, CD)    CD       one       one        CD   
  10        (of, IN)    IN        of        of        IN   
  11     (our, PRP$)  PRP$       our       our        PR   
  12     (inner, JJ)    JJ     inner     inner        JJ   
  13  (planets, NNS)   NNS   planets   planets        NN   
  15  (remember, VB)    VB  remember  remember        VB   
  17       (one, CD)    CD       one       one        CD   
  18        (of, IN)    IN        of        of        IN   
  19       (the, DT)    DT       the       the        DT   
  20     (ones, NNS)   NNS      ones      ones        NN   
  21       (we, PRP)   PRP        we        we        PR   
  22   (talked, VBD)   VBD    talked    talked        VB   
  23     (about, IN)    IN     about     about        IN   
  24     (today, NN)    NN     today     today        NN   

                           filename  classification                person  \
0 0   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  0   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  2   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  3   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  4   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  0   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  1   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  2   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  3   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  0   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  0   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  1   01_0101_G4_L03_trans

### Vocab Tables 

In [17]:
# Teacher Vocab 
df_tokens_teacher = df_tokens[df_tokens['classification'] == 1]
vocab_teacher = df_tokens_teacher['term_str'].value_counts().to_frame('n')
vocab_teacher.index.name = 'term_str'
vocab_teacher['p'] = vocab_teacher['n'] / vocab_teacher['n'].sum()  # Probability of each term
vocab_teacher['i'] = -np.log2(vocab_teacher['p'])  # Information content
vocab_teacher['n_chars'] = vocab_teacher.index.str.len()  # Number of characters in each term

In [18]:
#Student Vocab
df_tokens_student = df_tokens[df_tokens['classification'] == 2]
vocab_student = df_tokens_student['term_str'].value_counts().to_frame('n')
vocab_student.index.name = 'term_str'
vocab_student['p'] = vocab_student['n'] / vocab_student['n'].sum()  # Probability of each term
vocab_student['i'] = -np.log2(vocab_student['p'])  # Information content
vocab_student['n_chars'] = vocab_student.index.str.len()  # Number of characters in each term
vocab_student.head()

,n,p,i,n_chars
term_str,,,,
the,1022,0.046316,4.432358,3
it,780,0.035348,4.822207,2
i,594,0.026919,5.215218,1
and,491,0.022251,5.489958,3
to,419,0.018988,5.718731,2


In [19]:
vocab_teacher.head(20)

,n,p,i,n_chars
term_str,,,,
you,5145,0.041348,4.596054,3
the,4201,0.033761,4.888493,3
to,3289,0.026432,5.241576,2
it,3058,0.024575,5.346637,2
i,2567,0.020630,5.599142,1
and,2386,0.019175,5.704631,3
that,2209,0.017753,5.815832,4
a,1920,0.015430,6.018119,1
s,1898,0.015253,6.034745,1


In [20]:
vocab_student.head(20)

,n,p,i,n_chars
term_str,,,,
the,1022,0.046316,4.432358,3
it,780,0.035348,4.822207,2
i,594,0.026919,5.215218,1
and,491,0.022251,5.489958,3
to,419,0.018988,5.718731,2
a,404,0.018309,5.771326,1
you,401,0.018173,5.782079,3
s,349,0.015816,5.982454,1
like,333,0.015091,6.050159,4


In [21]:
df_tokens_student.head(50)

pos_tuple   pos token_str term_str pos_group  \
1  0      (This, DT)    DT      This     this        DT   
   1       (is, VBZ)   VBZ        is       is        VB   
   2      (easy, JJ)    JJ      easy     easy        JJ   
3  0        (I, PRP)   PRP         I        i        PR   
   1       (do, VBP)   VBP        do       do        VB   
   2       (n't, RB)    RB       n't       nt        RB   
   3      (know, VB)    VB      know     know        VB   
5  0    (______, NN)    NN    ______   ______        NN   
7  0      (Four, CD)    CD      Four     four        CD   
10 0        (I, PRP)   PRP         I        i        PR   
   1        (ca, MD)    MD        ca       ca        MD   
   2       (n't, RB)    RB       n't       nt        RB   
   3       (see, VB)    VB       see      see        VB   
   4       (it, PRP)   PRP        it       it        PR   
12 0     (Those, DT)    DT     Those    those        DT   
   1       (two, CD)    CD       two      two        CD   
   2     (three, CD)    CD     three    three        CD   
13 0       (Do, VBP)   VBP        Do       do        VB   
   1       (we, PRP)   PRP        we       we        PR   
   2       (hit, VB)    VB       hit      hit        VB   
   3     (done, VBN)   VBN      done     done        VB   
15 0   (Jupiter, NN)    NN   Jupiter  jupiter        NN   
17 0   (Giants, NNS)   NNS    Giants   giants        NN   
19 0  (Neptune, NNP)   NNP   Neptune  neptune        NN   
21 0  (Jupiter, NNP)   NNP   Jupiter  jupiter        NN   
   2   (Saturn, NNP)   NNP    Saturn   saturn        NN   
   4   (Uranus, NNP)   NNP    Uranus   uranus        NN   
   6  (Neptune, NNP)   NNP   Neptune  neptune        NN   
23 0      (Gas, NNP)   NNP       Gas      gas        NN   
   1   (giants, NNS)   NNS    giants   giants        NN   
28 0     (Earth, NN)    NN     Earth    earth        NN   
31 0     (Mars, NNS)   NNS      Mars     mars        NN   
33 0     (Our, PRP$)  PRP$       Our      our        PR   
   2     (fifth, JJ)    JJ     fifth    fifth        JJ   
   3    (planet, NN)    NN    planet   planet        NN   
35 0     (Our, PRP$)  PRP$       Our      our        PR   
   1    (sixth, NNS)   NNS     sixth    sixth        NN   
36 0   (Seventh, JJ)    JJ   Seventh  seventh        JJ   
   1    (planet, NN)    NN    planet   planet        NN   
37 0        (No, DT)    DT        No       no        DT   
40 0        (I, PRP)   PRP         I        i        PR   
   1       ('m, VBP)   VBP        'm        m        VB   
   2      (gon, VBG)   VBG       gon      gon        VB   
   3        (na, TO)    TO        na       na        TO   
   4       (get, VB)    VB       get      get        VB   
   5       (it, PRP)   PRP        it       it        PR   
   6     (wrong, JJ)    JJ     wrong    wrong        JJ   
   8        (oh, JJ)    JJ        oh       oh        JJ   
   9       (man, NN)    NN       man      man        NN   
42 0      (Why, WRB)   WRB       Why      why        WR   

                           filename  classification                person  \
1  0  01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   
   1  01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   
   2  01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   
3  0  01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   
   1  01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   
   2  01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   
   3  01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   
5  0  01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   
7  0  01_0101_G4_L03_transcript.txt             2.0           Whole Class   
10 0  01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   
   1  01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   
   2  01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   


In [22]:
df_tokens_teacher.head(50)

pos_tuple   pos token_str  term_str pos_group  \
0 0       (Nope, NN)    NN      Nope      nope        NN   
  0   (Alright, NNP)   NNP   Alright   alright        NN   
  2       (what, WP)    WP      what      what        WP   
  3       (are, VBP)   VBP       are       are        VB   
  4      (they, PRP)   PRP      they      they        PR   
  0       (What, WP)    WP      What      what        WP   
  1   (planets, NNS)   NNS   planets   planets        NN   
  2      (have, VBP)   VBP      have      have        VB   
  3     (rings, NNS)   NNS     rings     rings        NN   
  0        (Yep, NN)    NN       Yep       yep        NN   
  0        (One, CD)    CD       One       one        CD   
  1      (more, JJR)   JJR      more      more        JJ   
2 0        (All, DT)    DT       All       all        DT   
  1      (right, NN)    NN     right     right        NN   
  3        (it, PRP)   PRP        it        it        PR   
  4        ('s, VBZ)   VBZ        's         s        VB   
  5        (not, RB)    RB       not       not        RB   
  6        (gon, VB)    VB       gon       gon        VB   
  7         (na, RB)    RB        na        na        RB   
  8         (be, VB)    VB        be        be        VB   
  9        (one, CD)    CD       one       one        CD   
  10        (of, IN)    IN        of        of        IN   
  11     (our, PRP$)  PRP$       our       our        PR   
  12     (inner, JJ)    JJ     inner     inner        JJ   
  13  (planets, NNS)   NNS   planets   planets        NN   
  0       (Say, NNP)   NNP       Say       say        NN   
  1       (that, IN)    IN      that      that        IN   
  3        (it, PRP)   PRP        it        it        PR   
  4        ('s, VBZ)   VBZ        's         s        VB   
  5        (not, RB)    RB       not       not        RB   
  6        (gon, VB)    VB       gon       gon        VB   
  7         (na, RB)    RB        na        na        RB   
  8         (be, VB)    VB        be        be        VB   
  9        (one, CD)    CD       one       one        CD   
  10        (of, IN)    IN        of        of        IN   
  11     (our, PRP$)  PRP$       our       our        PR   
  12     (inner, JJ)    JJ     inner     inner        JJ   
  13  (planets, NNS)   NNS   planets   planets        NN   
  15  (remember, VB)    VB  remember  remember        VB   
  17       (one, CD)    CD       one       one        CD   
  18        (of, IN)    IN        of        of        IN   
  19       (the, DT)    DT       the       the        DT   
  20     (ones, NNS)   NNS      ones      ones        NN   
  21       (we, PRP)   PRP        we        we        PR   
  22   (talked, VBD)   VBD    talked    talked        VB   
  23     (about, IN)    IN     about     about        IN   
  24     (today, NN)    NN     today     today        NN   
4 0        (Not, RB)    RB       Not       not        RB   
  1        (gon, VB)    VB       gon       gon        VB   
  2         (na, RB)    RB        na        na        RB   

                           filename  classification           person  \
0 0   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  0   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  2   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  3   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  4   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  0   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  1   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  2   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  3   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  0   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  0   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  1   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
2 0   01_0101_